In [1]:
data_cuaca = [
    {'id': 'C1', 'kota': 'Jakarta', 'suhu': 35, 'keterangan': 'Cerah dan panas terik'},
    {'id': 'C2', 'kota': 'Bandung', 'suhu': 24, 'keterangan': 'Berawan dan sejuk'},
    {'id': 'C3', 'kota': 'Surabaya', 'suhu': 34, 'keterangan': 'Panas dengan sedikit angin'},
    {'id': 'C4', 'kota': 'Medan', 'suhu': 29, 'keterangan': 'Lembab dan agak mendung'},
    {'id': 'C5', 'kota': 'Bogor', 'suhu': 23, 'keterangan': 'Hujan ringan'},
]


In [ ]:
# Buat closure

def buat_pendeteksi_suhu(batas, jenis):
    jenis_norm = str(jenis).lower()
    if jenis_norm not in {"tinggi", "rendah"}:
        raise ValueError("jenis harus 'tinggi' atau 'rendah'")

    def pendeteksi(entry):
        suhu = entry.get("suhu")
        if suhu is None:
            raise KeyError("data tidak memiliki key 'suhu'")
        return suhu > batas if jenis_norm == "tinggi" else suhu < batas

    return pendeteksi

buat_pendeteksi_suhu_alias = buat_pendeteksi_suhu

panas = buat_pendeteksi_suhu(30, "tinggi")
dingin = buat_pendeteksi_suhu(25, "rendah")

kota_suhu_tinggi = [catatan["kota"] for catatan in data_cuaca if panas(catatan)]
kota_suhu_rendah = [catatan["kota"] for catatan in data_cuaca if dingin(catatan)]

print(f"Kota dengan suhu tinggi = {kota_suhu_tinggi}")
print(f"Kota dengan suhu rendah = {kota_suhu_rendah}")

Kota dengan suhu tinggi = ['Jakarta', 'Surabaya']
Kota dengan suhu rendah = ['Bandung', 'Bogor']


In [ ]:
# Membersihkan argumen teks

from functools import wraps


def sanitasi_teks(func):
    @wraps(func)
    def wrapper(teks, *args, **kwargs):
        teks_bersih = "" if teks is None else str(teks).lower().strip()
        return func(teks_bersih, *args, **kwargs)

    return wrapper


@sanitasi_teks
def hitung_jumlah_kata(teks):
    print(f"DEBUG: Menghitung kata dari: '{teks}'")
    return len(teks.split())


for catatan in data_cuaca:
    jumlah_kata = hitung_jumlah_kata(catatan["keterangan"])
    print(f"Jumlah kata = {jumlah_kata}")

DEBUG: Menghitung kata dari: 'cerah dan panas terik'
Jumlah kata = 4
DEBUG: Menghitung kata dari: 'berawan dan sejuk'
Jumlah kata = 3
DEBUG: Menghitung kata dari: 'panas dengan sedikit angin'
Jumlah kata = 4
DEBUG: Menghitung kata dari: 'lembab dan agak mendung'
Jumlah kata = 4
DEBUG: Menghitung kata dari: 'hujan ringan'
Jumlah kata = 2
